In [1]:
import requests
import json
import time

API_KEY = "456a129b2990178516e5e69f5a8d5a74"
BASE_URL = "https://api.themoviedb.org/3"


In [2]:
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Function to fetch movie details
def get_movie_details(movie_id):
    url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}&language=en-US"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# Function to fetch movie credits (director, cast)
def get_movie_credits(movie_id):
    url = f"{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        credits = response.json()
        # Extract director name
        director = next((crew["name"] for crew in credits["crew"] if crew["job"] == "Director"), "Unknown")
        # Extract top 5 cast members
        cast = [actor["name"] for actor in credits["cast"][:5]]
        return director, cast
    return "Unknown", []

# Function to save data to JSON
def save_to_json(data, filename="movies.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

# Function to fetch and store multiple movies
def fetch_and_store_movies(num_movies=100):
    all_movies = []
    
    for page in range(1, (num_movies // 20) + 2):  # Each API call returns 20 movies per page
        url = f"{BASE_URL}/movie/popular?api_key={API_KEY}&language=en-US&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            movies = response.json()["results"]
            
            for movie in movies:
                movie_id = movie["id"]
                details = get_movie_details(movie_id)
                director, cast = get_movie_credits(movie_id)
                
                movie_data = {
                    "title": details["title"],
                    "description": details["overview"],
                    "director": director,
                    "cast": cast,
                    "poster_url": f"https://image.tmdb.org/t/p/w500{details['poster_path']}" if details["poster_path"] else "N/A"
                }
                
                all_movies.append(movie_data)
                
                # Stop when we reach the required number of movies
                if len(all_movies) >= num_movies:
                    break
            
            time.sleep(1)  # To avoid hitting rate limits
        
        if len(all_movies) >= num_movies:
            break
    
    save_to_json(all_movies)
    print(f"Saved {len(all_movies)} movies to movies.json")


In [4]:
fetch_and_store_movies(5000)

KeyboardInterrupt: 